### Importing the required libraries

In [4]:
import pydap
import netCDF4 as nc
import pandas as pd
import h5py
import xarray as xr
import geopy.distance

In [5]:
file = 'oco2_LtCO2_220101_B11014Ar_220728214731s.nc4'
ds = nc.Dataset(file)

FileNotFoundError: [Errno 2] No such file or directory: b'oco2_LtCO2_220101_B11014Ar_220728214731s.nc4'

In [3]:
print(ds)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    filter_function: oco2_xco2_quality_flag_b11
    bc_function: oco2_xco2_bias_correct_b11
    l2_reader: read_oco_l2_b11
    lite_definition_module: lite_file_definition_oco2_b11
    filtering state: Filtered with oco2_lite_file_prefilter_b11
    warn_level_function: None
    Bias_Correction_land: XCO2_Bias_Corrected = (XCO2_Raw + 0.830*(dpfrac + 0.0) + 0.0330*(co2_grad_del + 5.00) + 0.270*((logDWS>(-5.0)) + 5.00) - 6.70*(aod_fine - 0.0300) + 33.0*(aod_ic - 0.00600) - 0.480*(albedo_quad_wco2*1e6 + 0.0600) - footprint_bias)/0.99970
    Bias_Correction_landTG: XCO2_Bias_Corrected = (XCO2_Raw + 0.770*(dpfrac + 0.300) + 0.0250*(co2_grad_del + 0.0) + 0.250*((logDWS>(-5.0)) + 5.00) - footprint_bias)/0.99970
    Bias_Correction_oceanGL: XCO2_Bias_Corrected = (XCO2_Raw + 0.250*(dp_sco2 + 0.0) - 8.00*(sqrt(albedo_wco2) - 0.280) + 0.0175*(co2_grad_del + 15.0) + 2.00*(max_declocking_2 + 0.0) + 14.0*(aod_wa - 0

In [4]:
print(ds.__dict__)

{'filter_function': 'oco2_xco2_quality_flag_b11', 'bc_function': 'oco2_xco2_bias_correct_b11', 'l2_reader': 'read_oco_l2_b11', 'lite_definition_module': 'lite_file_definition_oco2_b11', 'filtering state': 'Filtered with oco2_lite_file_prefilter_b11', 'warn_level_function': 'None', 'Bias_Correction_land': 'XCO2_Bias_Corrected = (XCO2_Raw + 0.830*(dpfrac + 0.0) + 0.0330*(co2_grad_del + 5.00) + 0.270*((logDWS>(-5.0)) + 5.00) - 6.70*(aod_fine - 0.0300) + 33.0*(aod_ic - 0.00600) - 0.480*(albedo_quad_wco2*1e6 + 0.0600) - footprint_bias)/0.99970', 'Bias_Correction_landTG': 'XCO2_Bias_Corrected = (XCO2_Raw + 0.770*(dpfrac + 0.300) + 0.0250*(co2_grad_del + 0.0) + 0.250*((logDWS>(-5.0)) + 5.00) - footprint_bias)/0.99970', 'Bias_Correction_oceanGL': 'XCO2_Bias_Corrected = (XCO2_Raw + 0.250*(dp_sco2 + 0.0) - 8.00*(sqrt(albedo_wco2) - 0.280) + 0.0175*(co2_grad_del + 15.0) + 2.00*(max_declocking_2 + 0.0) + 14.0*(aod_wa - 0.0100) - 2.00*(err - 0.460) - footprint_bias)/0.99970', 'Bias_Correction_ocean

In [5]:
for dim in ds.variables.values():
    print(dim)

<class 'netCDF4._netCDF4.Variable'>
uint64 sounding_id(sounding_id)
    units: YYYYMMDDhhmmssmf
    long_name: OCO2 Sounding ID
    missing_value: 0
    comment: from scan time in UTC
unlimited dimensions: 
current shape = (135096,)
filling on, default _FillValue of 18446744073709551614 used
<class 'netCDF4._netCDF4.Variable'>
int16 levels(levels)
unlimited dimensions: 
current shape = (20,)
filling on, default _FillValue of -32767 used
<class 'netCDF4._netCDF4.Variable'>
int16 bands(bands)
unlimited dimensions: 
current shape = (3,)
filling on, default _FillValue of -32767 used
<class 'netCDF4._netCDF4.Variable'>
int16 vertices(vertices)
unlimited dimensions: 
current shape = (4,)
filling on, default _FillValue of -32767 used
<class 'netCDF4._netCDF4.Variable'>
int16 footprints(footprints)
unlimited dimensions: 
current shape = (8,)
filling on, default _FillValue of -32767 used
<class 'netCDF4._netCDF4.Variable'>
int16 date(sounding_id, epoch_dimension)
    units: none
    long_name: 

In [6]:
df = pd.read_hdf("oco2_L2StdGL_45213a_230101_B11008_230103045740.h5")

ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.

In [7]:
with h5py.File('oco2_L2StdGL_45213a_230101_B11008_230103045740.h5', 'r') as f:
    print(f["Dimensions"].keys())

<KeysViewHDF5 ['AerosolParam', 'AerosolSubColumn', 'AerosolType', 'AlbedoWavelength', 'AncFile', 'EuclidDim', 'Frame', 'Gap', 'InputPtr', 'L2FullPhysicsInputPtr', 'Layer', 'Level', 'MetLevel', 'O2Slice', 'OrbitParamPtr', 'Retrieval', 'Sounding', 'SoundingPosition', 'Spectrum', 'StrongCO2Slice', 'Vertex', 'WeakCO2Slice']>


In [8]:
with h5py.File('oco2_LtCO2_220101_B11014Ar_220728214731s.nc4', 'r') as f:
    print(f.keys())

<KeysViewHDF5 ['Meteorology', 'Preprocessors', 'Retrieval', 'Sounding', 'bands', 'co2_profile_apriori', 'date', 'epoch_dimension', 'file_index', 'footprints', 'latitude', 'levels', 'longitude', 'pressure_levels', 'pressure_weight', 'sensor_zenith_angle', 'solar_zenith_angle', 'sounding_id', 'source_files', 'time', 'vertex_latitude', 'vertex_longitude', 'vertices', 'xco2', 'xco2_apriori', 'xco2_averaging_kernel', 'xco2_qf_bitflag', 'xco2_qf_simple_bitflag', 'xco2_quality_flag', 'xco2_uncertainty']>


In [9]:
with h5py.File('oco2_LtCO2_220101_B11014Ar_220728214731s.nc4', 'r') as f:
    print(f["latitude"])

<HDF5 dataset "latitude": shape (135096,), type "<f4">


In [10]:
import numpy as np
import pandas as pd

# Load the XCO2 data from OCO-2 and GOSAT
df_oco2 = pd.read_csv("df_oco2.csv")
df_gosat = pd.read_csv("df_gosat.csv")

# Align the data in terms of time and location
df_oco2.sort_values(by=["time", "latitude", "longitude"], inplace=True)
df_gosat.sort_values(by=["time", "latitude", "longitude"], inplace=True)

# Merge the data by taking the average of the XCO2 values for a given location and time
combined_data = pd.DataFrame(columns=["time", "latitude", "longitude", "xco2"])
oco2_index = 0
gosat_index = 0
while oco2_index < len(df_oco2) and gosat_index < len(df_gosat):
    oco2_time = df_oco2.iloc[oco2_index]["time"]
    gosat_time = df_gosat.iloc[gosat_index]["time"]
    if oco2_time < gosat_time:
        oco2_index += 1
    elif oco2_time > gosat_time:
        gosat_index += 1
    else:
        oco2_latitude = df_oco2.iloc[oco2_index]["latitude"]
        gosat_latitude = df_gosat.iloc[gosat_index]["latitude"]
        oco2_longitude = df_oco2.iloc[oco2_index]["longitude"]
        gosat_longitude = df_gosat.iloc[gosat_index]["longitude"]
        if abs(oco2_latitude - gosat_latitude) < 1e-6 and abs(oco2_longitude - gosat_longitude) < 1e-6:
            combined_xco2 = (df_oco2.iloc[oco2_index]["xco2"] + df_gosat.iloc[gosat_index]["xco2"]) / 2
            combined_data = combined_data.append({"time": oco2_time, "latitude": oco2_latitude, "longitude": oco2_longitude, "xco2": combined_xco2}, ignore_index=True)
            oco2_index += 1
            gosat_index += 1

# Save the combined data to a CSV file
combined_data.to_csv("combined_data.csv", index=False)



FileNotFoundError: [Errno 2] No such file or directory: 'df_oco2.csv'

In [11]:
f_gosat = nc.Dataset("gosat_2016_ja1.nc4")
f_oco2 = nc.Dataset("oco2_2016_jan1.nc4")

In [12]:
f_oco2["time"][0]

masked_array(data=1.4516064e+09,
             mask=False,
       fill_value=1e+20)

In [13]:
f_gosat.keys()

AttributeError: NetCDF: Attribute not found

In [14]:
df_oco2 = pd.DataFrame(columns=["xco2","latitude","longitude","time"])

In [15]:
df_oco2["xco2"] = f_oco2["xco2"][:10000000]
df_oco2["latitude"] = f_oco2["latitude"][:10000000]
df_oco2["longitude"] = f_oco2["longitude"][:10000000]
df_oco2["time"] = f_oco2["time"][:10000000]

In [16]:
df_oco2.head()

,xco2,latitude,longitude,time
0,400.449890,-3.628539,-157.433411,1.451606e+09
1,400.224304,-3.639151,-157.430817,1.451606e+09
2,400.457947,-3.649849,-157.428299,1.451606e+09
3,399.878815,-3.660598,-157.425873,1.451606e+09
4,400.196655,-3.671286,-157.423538,1.451606e+09


In [17]:
df_oco2.describe()

/home/nikhil/anaconda3/envs/tri_nit/lib/python3.11/site-packages/numpy/lib/function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
/home/nikhil/anaconda3/envs/tri_nit/lib/python3.11/site-packages/numpy/lib/function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
/home/nikhil/anaconda3/envs/tri_nit/lib/python3.11/site-packages/numpy/lib/function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
/home/nikhil/anaconda3/envs/tri_nit/lib/python3.11/site-packages/numpy/lib/function_base.py:4737: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


,xco2,latitude,longitude,time
count,166637.000000,166637.000000,166637.000000,1.666370e+05
mean,399.110199,-10.815790,5.014174,1.451649e+09
std,5.986735,34.866135,99.591522,2.324266e+04
min,329.732452,-84.980553,-179.727386,1.451606e+09
25%,398.553253,-33.347469,-74.581886,1.451631e+09
50%,400.140289,-7.414774,21.309219,1.451648e+09
75%,401.546051,18.839664,86.354759,1.451664e+09
max,416.550964,54.536709,174.433411,1.451690e+09


In [18]:
df_gosat.describe()

NameError: name 'df_gosat' is not defined

In [19]:
df_gosat = pd.DataFrame(columns=["xco2","latitude","longitude","time"])
df_gosat["xco2"] = f_gosat["xco2"][:10000000]
df_gosat["latitude"] = f_gosat["latitude"][:10000000]
df_gosat["longitude"] = f_gosat["longitude"][:10000000]
df_gosat["time"] = f_gosat["time"][:10000000]
df_gosat.head()

,xco2,latitude,longitude,time
0,401.641479,-74.992409,162.841263,1.451608e+09
1,405.424896,-74.985619,162.859528,1.451608e+09
2,410.683899,-74.977623,162.877975,1.451608e+09
3,404.740845,-74.412048,141.735397,1.451608e+09
4,409.490906,-77.648460,125.245255,1.451609e+09


In [20]:
combined_data = pd.DataFrame(columns=["time", "latitude", "longitude", "xco2"])
oco2_index = 0
gosat_index = 0
diff1 =[]
diff2 =[]
while oco2_index < len(df_oco2):
    oco2_latitude = df_oco2.iloc[oco2_index]["latitude"]
    oco2_longitude = df_oco2.iloc[oco2_index]["longitude"]
    while gosat_index < len(df_gosat):
        gosat_latitude = df_gosat.iloc[gosat_index]["latitude"]
        gosat_longitude = df_gosat.iloc[gosat_index]["longitude"]
        diff1.append(abs(oco2_latitude - gosat_latitude))
        diff2.append(abs(oco2_longitude - gosat_longitude))
        if abs(oco2_latitude - gosat_latitude) < 1 and abs(oco2_longitude - gosat_longitude) < 1:
            combined_xco2 = (df_oco2.iloc[oco2_index]["xco2"] + df_gosat.iloc[gosat_index]["xco2"]) / 2
            combined_data = combined_data.append({"time": oco2_time, "latitude": oco2_latitude, "longitude": oco2_longitude, "xco2": combined_xco2},ignore_index=True)
        gosat_index += 1
    oco2_index += 1
    

# Save the combined data to a CSV file
combined_data.to_csv("combined_data.csv", index=False)
print(min(diff1))
print(min(diff2))


0.7831883430480957
2.7499237060546875


In [21]:
combined_data

,time,latitude,longitude,xco2


In [22]:
f_oco2["time"]

<class 'netCDF4._netCDF4.Variable'>
float64 time(sounding_id)
    units: seconds since 1970-01-01 00:00:00
    long_name: time
    missing_value: -999999.0
unlimited dimensions: 
current shape = (166637,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [25]:
xr.read_Dataset()

AttributeError: module 'xarray' has no attribute 'read_Dataset'

In [28]:
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print (geopy.distance.geodesic(coords_1, coords_2).km)

279.35290160430094
